## ANLY-512 Project - Data Collection

**Authors**: Victor De Lima, Matthew Moriarty

This Jupyter notebook provides the data collection steps for our ANLY-512 project: Community Prediction with NLP. Due to the immense size of data (around 20 GB), we have employed Google Colab for retrieving the data and storing it off into 20 separate "chunks" used in our analysis.

In [ ]:
# install transformers on colab
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 6.7 MB/s eta 0:00:00


In [ ]:
# install datasets on colab
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 7.6 MB/s eta 0:00:00


In [ ]:
# import libraries
from datasets import load_dataset
import pandas as pd
import numpy as np

In [ ]:
# load the reddit dataset
dataset = load_dataset("reddit")

# the loaded dataset is a JSON, so we have to extract the 'train' dataset
df = dataset['train']
df.shape

Generating train split:   0%|          | 0/3848330 [00:00<?, ? examples/s]

Dataset reddit downloaded and prepared to /root/.cache/huggingface/datasets/reddit/default/1.0.0/98ba5abea674d3178f7588aa6518a5510dc0c6fa8176d9653a3546d5afcb3969. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

(3848330, 8)

In [ ]:
# collect the number of occurrences for each subreddit
subreddits = pd.Series(df['subreddit']).value_counts()

# show the top ten subreddits
subreddits.index[:10]

Index(['AskReddit', 'relationships', 'leagueoflegends', 'tifu',
       'relationship_advice', 'trees', 'gaming', 'atheism', 'AdviceAnimals',
       'funny'],
      dtype='object')

In [ ]:
# display the features of the dataset
df.features

{'author': Value(dtype='string', id=None),
 'body': Value(dtype='string', id=None),
 'normalizedBody': Value(dtype='string', id=None),
 'subreddit': Value(dtype='string', id=None),
 'subreddit_id': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'content': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None)}

In [ ]:
# remove unneeded columns, including extra 'body' features and IDs
dataset = dataset.remove_columns(['body', 'normalizedBody', 'subreddit_id', 'id'])
# pick the top 5 subreddits (relationships and relationship_advice will be combined)
top5 = ['AskReddit', 'relationships', 'leagueoflegends', 'tifu', 'relationship_advice', 'trees']#, 'gaming', 'atheism']
# extract rows only from top 5 subreddits
top5df = dataset.filter(lambda example: example['subreddit'] in top5)

In [ ]:
# convert to pandas dataframe for easier use (takes several minutes)
top5dfp = pd.DataFrame(top5df['train'])
top5dfp.head()

,author,subreddit,content,summary
0,phyzishy,AskReddit,"Yeah, but most folks think avoiding gluten wil...",stupid stuff.
1,Perservere,leagueoflegends,Didn't they lose 6 games in a row? Just becaus...,"just because you're close ""at times"" doesn't m..."
2,fallsuspect,AskReddit,You probably won't come off as an ass if you j...,"just get both of their numbers, text the one y..."
3,Buck_Speedjunk,trees,"This picture doesn't follow too well, as defin...",It's a half-assed fan-art that literally put e...
4,SinglesRazor,AskReddit,"I want to say this was about two weeks ago, co...",Fuck Slender Man.


In [ ]:
# combine 'relationship_advice' and 'relationships'
top5dfp['subreddit'] = top5dfp['subreddit'].replace('relationship_advice', 'relationships')

In [ ]:
# sample 40,000 posts from each category
dfsample = top5dfp.groupby('subreddit').sample(n = 40000, replace = False, random_state = 1)

In [ ]:
# export to CSV (300+ MB)
dfsample.to_csv('df40k.csv')

In [ ]:
# split the 300 MB dataset into 20 pieces, each (10000, 4) in shape
df_split = np.array_split(dfsample, 20)

In [ ]:
# export the 20 datasets to CSV for use in the analysis
for i in range(20):
  filename = 'df40k_' + str(i) + '.csv'
  df_split[i].to_csv(filename)